In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# value object
# takes a single scalar value that it wraps and keep tracks off
class Value:
    def __init__(self,data, _children=(),_op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label= label
        
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self,other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward =  _backward   #we try to save these fxn not call the fxn bcz these fxn's return none, just want to store the fxn
        return out

    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self,other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other,(int,float)),"only support int/float powers for now"
        out = Value(self.data**other, (self, ), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward
        return out

   
    def tanh(self):
        n = self.data
        t = (math.exp(2*n)-1) / (math.exp(2*n) +1)
        out = Value(t,(self,),'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
            
        return out 

    def exp(self):
        x = self.data
        out = Value(math.exp(x),(self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad

        out._backward= _backward
        return out

    def backward(self):
        # made a graph of forward prop
        topo = []
        visited =set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        # set grad for output
        self.grad = 1.0

        #call _backward() in reverse of topo[]
        for node in reversed(topo):
            node._backward()
        
    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self,other):
        return self - other

    def __radd__(self,other):
        return self + other  
        
    def __rmul__(self, other):   #other * self
        return self * other

    def __truediv__(self,other):   # a / b 
        return self * other**-1

    def __rtruediv__(self,other):
        return other * self**-1

    def __repr__(self):       # wrapper function
        return f"Value(data={self.data})"
    


In [5]:
# input x1,x2
x1 = Value(2.0)
x2 = Value(0.0)
# weights 
w1 = Value(-3.0)
w2 = Value(1.0)
#bias of the neuron
b = Value(6.8813735870195432)
#neuron = x1w1 + x2w2 + b
x1w1 = x1*w1
x2w2 = x2*w2
x1w1x2w2 = x1w1 + x2w2
n = x1w1x2w2 + b
#activation fxn
#o = n.tanh()  #exp(2z)-1 / exp(2z)+1
e = (2*n).exp()
o = (e - 1) / (e + 1)
o.backward()